In [1]:
import io
import numpy as np
import tensorflow as tf
import pandas as pd
from scipy.io import wavfile
from python_speech_features import mfcc
import matplotlib.pyplot as plt
!pip install umap-learn
import umap
import seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pydub import AudioSegment


import warnings
warnings.filterwarnings("ignore")

import tensorflow_addons as tfa

In [2]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(
        256,
        input_shape=(32,13),
        return_sequences=True
    ),
    tf.keras.layers.LSTM(
        256
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        256, 
        activation=None
    ), 
    tf.keras.layers.Lambda(
        lambda x: tf.math.l2_normalize(x, axis=1)
    )
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 32, 256)           276480    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               65792     
_________________________________________________________________
lambda (Lambda)              (None, 256)               0         
Total params: 867,584
Trainable params: 867,584
Non-trainable params: 0
_________________________________________________________________


In [3]:
!ls /data/UrbanSound8K/processed_data/

preprocessed_data.npy  processed_data.npy  processed_labels.json


In [4]:
ProcessedData = np.load(
    "/data/UrbanSound8K/processed_data/processed_data.npy",
    allow_pickle=True
)

In [5]:
metadata_path = "/data/UrbanSound8K/metadata/UrbanSound8K.csv"

df = pd.read_csv(metadata_path)

In [6]:
df.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [7]:
filepath = "/data/UrbanSound8K/audio/fold{fold}/{filename}"

len(df.index)

8732

In [ ]:
test_file = filepath.format(
    fold=5,
    filename="100032-3-0-0.wav"
)

fs, sample = wavfile.read(test_file)

Data = []
Labels = []
Lengths = []

exception_count = 0
idx = 0
for row in tqdm(df.iterrows()):
    row=row[1]
    filename = filepath.format(
        fold=row['fold'],
        filename=row['slice_file_name']
    )
    try:
        audio = AudioSegment.from_wav(filename)
        fs, sample = audio.frame_rate, \
            np.array(audio.get_array_of_samples(), dtype=float)
        data = mfcc(signal=sample, samplerate=fs)
        Lengths.append(data.shape[0])
        Data.append(data)
        Labels.append(row['class'])
        idx += 1
    except Exception as e:
        print(f"Exception: {filename}")
        exception_count += 1

print(f"Number of Exceptions: {exception_count}")


0it [00:00, ?it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

3it [00:00, 25.12it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

6it [00:00, 23.57it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

9it [00:00, 23.88it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

14it [00:00, 27.51it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

17it [00:00, 26.74it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

20it [00:00, 26.56it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

23it [00:00, 24.54it/s]WARNING:root:frame


68it [00:02, 17.74it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

70it [00:03, 11.19it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

72it [00:03,  8.93it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

74it [00:03,  7.62it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

76it [00:04,  7.14it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

77it [00:04,  6.44it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

78it [00:04,  6.41it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

79it [00:04,  6.38it/s]WARNING:ro


131it [00:08, 20.66it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

134it [00:08, 21.78it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

137it [00:08, 22.71it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

140it [00:08, 22.77it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

143it [00:09, 23.66it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

146it [00:09, 23.54it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

149it [00:09, 24.26it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

152it [00:09, 24.39it/s]WA


197it [00:13,  8.23it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

200it [00:14, 10.28it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

203it [00:14, 12.60it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

206it [00:14, 14.48it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

209it [00:14, 16.91it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

212it [00:14, 19.02it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

215it [00:14, 20.50it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

218it [00:16,  4.16it/s]WA


258it [00:23,  5.64it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

259it [00:23,  6.17it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

261it [00:23,  7.65it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

265it [00:23, 10.01it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

269it [00:23, 12.79it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

272it [00:23, 14.88it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

275it [00:24, 16.52it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

278it [00:24, 17.23it/s]WA

...


390it [01:12, 20.47it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

393it [01:12, 21.06it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

396it [01:12, 21.44it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

399it [01:12, 21.71it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

402it [01:12, 20.76it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

405it [01:12, 19.68it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

408it [01:12, 20.04it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

411it [01:13, 19.83it/s]WA


457it [01:15, 21.28it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

460it [01:15, 17.44it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

462it [01:15, 15.16it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

464it [01:16, 13.63it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

466it [01:16, 13.17it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

468it [01:16, 12.66it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

470it [01:16, 12.35it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

472it [01:16, 12.17it/s]WA


522it [01:19, 21.45it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

525it [01:19, 23.20it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

528it [01:19, 24.19it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

531it [01:19, 24.95it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

534it [01:19, 25.02it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

537it [01:19, 24.99it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

540it [01:20, 23.75it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

543it [01:20, 24.15it/s]WA


587it [01:23, 12.15it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

591it [01:23, 15.16it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

594it [01:23, 16.41it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

597it [01:24, 17.45it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

600it [01:24, 10.52it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

602it [01:25,  4.04it/s]WARNING:root:frame length (1103) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

604it [01:26,  4.97it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

606it [01:26,  5.19it/s]WA

653it [01:29, 18.95it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

655it [01:29, 17.13it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

657it [01:29, 17.07it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

659it [01:29, 16.22it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

661it [01:29, 16.85it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

663it [01:29, 17.59it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

666it [01:29, 18.68it/s]WARNING:root:frame length (1200) is greater than FFT size (512), frame will be truncated. Increase NFFT to avoid.

669it [01:29, 19.36it/s]WAR

In [ ]:
Data_ = np.array(Data)

np.save("/home/jovyan/work/preprocessed_data.npy", Data_)
import json

In [ ]:
plt.hist(Lengths)

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_wav(filename)

audio.frame_rate

print(type(audio.get_array_of_samples()))

_data = mfcc(
    signal=np.array(audio.get_array_of_samples(),dtype=float), 
    samplerate=audio.frame_rate,
    nfft=2048+512
)

print(_data.shape)

In [ ]:
_Data = tf.keras.preprocessing.sequence.pad_sequences(
    Data,
    maxlen=1024,
    dtype='float32',
    padding='pre',
    truncating='pre',
    value=0.0
)

from random import randint

Map = {}
l = 0
Label = []

for label in Labels:
    if label not in Map:
        Map[label] = l
        l += 1
    Label.append(Map[label])

In [ ]:
np.save("/home/jovyan/work/processed_data.npy", _Data)
import json
json.dump(Label, open("/home/jovyan/work/processed_labels.json", 'w'))

In [ ]:
plt.hist(Lengths)

In [ ]:
from pydub import AudioSegment

audio = AudioSegment.from_wav(filename)

audio.frame_rate

print(type(audio.get_array_of_samples()))

_data = mfcc(
    signal=np.array(audio.get_array_of_samples(),dtype=float), 
    samplerate=audio.frame_rate,
    nfft=2048+512
)

print(_data.shape)

In [ ]:
_Data = tf.keras.preprocessing.sequence.pad_sequences(
    Data,
    maxlen=1024,
    dtype='float32',
    padding='pre',
    truncating='pre',
    value=0.0
)

from random import randint

Map = {}
l = 0
Label = []

for label in Labels:
    if label not in Map:
        Map[label] = l
        l += 1
    Label.append(Map[label])

In [ ]:
np.save("/home/jovyan/work/processed_data.npy", _Data)
import json
json.dump(Label, open("/home/jovyan/work/processed_labels.json", 'w'))

In [ ]:
_Data.shape

In [ ]:
X = zip(_Data, Label)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((_Data, Label))
dataset = dataset.shuffle(1024).batch(32)
dataset

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())


In [ ]:
# Train the network
history = model.fit(
    dataset,
    epochs=40
)

In [ ]:
model.save("urban_sounds.tf")

In [ ]:
X = model.predict(_Data)

In [ ]:
X.shape

In [ ]:
import umap

reducer = umap.UMAP()
embedding = reducer.fit_transform(X)
embedding.shape

In [ ]:
plt.scatter(embedding[:,0], embedding[:,1], c=[sns.color_palette()[x] for x in Label])